In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt

In [2]:
dotenv_path = os.path.join(os.getcwd(), '../../credentials.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
#print(os.getenv("HOST"))
host=os.getenv("HOST")
user=os.getenv("USER")
password=os.getenv("PASSWORD")
port=os.getenv("PORT")
database=os.getenv("DATABASE")
dbms=os.getenv("DBMS")
# Conectar a PostgreSQL
engine = create_engine(f'{dbms}://{user}:{password}@{host}:{port}/{database}')

In [4]:
query = 'SELECT * FROM raw_data_candidates'
df = pd.read_sql(query, engine)

In [7]:
df = pd.DataFrame(df)

In [8]:
df.head()

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


In [9]:
df.columns

Index(['First Name', 'Last Name', 'Email', 'Application Date', 'Country',
       'YOE', 'Seniority', 'Technology', 'Code Challenge Score',
       'Technical Interview Score'],
      dtype='object')

In [10]:
df.describe()

,YOE,Code Challenge Score,Technical Interview Score
count,50000.000000,50000.000000,50000.000000
mean,15.286980,4.996400,5.003880
std,8.830652,3.166896,3.165082
min,0.000000,0.000000,0.000000
25%,8.000000,2.000000,2.000000
50%,15.000000,5.000000,5.000000
75%,23.000000,8.000000,8.000000
max,30.000000,10.000000,10.000000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   First Name                 50000 non-null  object
 1   Last Name                  50000 non-null  object
 2   Email                      50000 non-null  object
 3   Application Date           50000 non-null  object
 4   Country                    50000 non-null  object
 5   YOE                        50000 non-null  int64 
 6   Seniority                  50000 non-null  object
 7   Technology                 50000 non-null  object
 8   Code Challenge Score       50000 non-null  int64 
 9   Technical Interview Score  50000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 3.8+ MB


In [18]:
"""There's a problem with the name of the countries,
 'cause there are more number of countries than the ones that exist in the world.
 The number of countries in the world it's 195"""
unique_Countries = df['Country'].unique().tolist()
print(len(unique_Countries))

244


In [16]:
df['Seniority'].unique().tolist()

['Intern', 'Mid-Level', 'Trainee', 'Junior', 'Lead', 'Architect', 'Senior']

In [17]:
df['Technology'].unique().tolist()

['Data Engineer',
 'Client Success',
 'QA Manual',
 'Social Media Community Management',
 'Adobe Experience Manager',
 'Sales',
 'Mulesoft',
 'DevOps',
 'Development - CMS Backend',
 'Salesforce',
 'System Administration',
 'Security',
 'Game Development',
 'Development - CMS Frontend',
 'Security Compliance',
 'Development - Backend',
 'Design',
 'Business Analytics / Project Management',
 'Development - Frontend',
 'Development - FullStack',
 'Business Intelligence',
 'Database Administration',
 'QA Automation',
 'Technical Writing']

# Recoding the dataframe

In [19]:
#First we need to make a copy of the original dataframe
df_Copy = df.copy()

In [23]:
df_Copy.columns

Index(['First Name', 'Last Name', 'Email', 'Application Date', 'Country',
       'YOE', 'Seniority', 'Technology', 'Code Challenge Score',
       'Technical Interview Score'],
      dtype='object')

In [24]:
# Rename Columns
df_Copy.rename(columns={'First Name': 'Name',
                         'Last Name': 'Last_Name',
                         'Application Date' : 'Date',
                         'Code Challenge Score' : 'Code_Score',
                         'Technical Interview Score' : 'Interview_Score'}, inplace=True)

In [25]:
#Confirm the changes
df_Copy.columns

Index(['Name', 'Last_Name', 'Email', 'Date', 'Country', 'YOE', 'Seniority',
       'Technology', 'Code_Score', 'Interview_Score'],
      dtype='object')

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

def get_Country(place):
    geolocator = Nominatim(user_agent="my_app")
    try:
        location = geolocator.geocode(place, exactly_one=True, language="en")
        if location:
            if 'address' in location.raw:
                address = location.raw['address']
                country = address.get('country', '')
                if country:
                    return country
            #if no country is found, we return the last part of the display
            return location.raw.get('display_name', '').split(', ')[-1]
        else:
            return place  #if no location is found, we return the original place
    except (GeocoderTimedOut, GeocoderUnavailable, KeyError, AttributeError):
        return place  #In case of any error, we return the original place

#we are going to use the unique values of the column 'Country'
unique_Countries = df_Copy['Country'].unique().tolist()

#Dictionary to store the normalized countries
Normalized_Countries = {}

#process each unique country
for place in unique_Countries:
    pais = get_Country(place)
    Normalized_Countries[place] = pais
    print(f"Procesado: {place} -> {pais}")  #print the progress
    time.sleep(1)  # Pause for 1 second to avoid geopy's rate limit

#update the column 'Country' with the normalized values
df_Copy['Country_Normalized'] = df_Copy['Country'].map(Normalized_Countries)

# print the results
print("\nFinal Results:")
for place, pais in Normalized_Countries.items():
    print(f"{place}: {pais}")

In [32]:
print(Normalized_Countries)

{'Norway': 'Norway', 'Panama': 'Panama', 'Belarus': 'Belarus', 'Eritrea': 'Eritrea', 'Myanmar': 'Myanmar', 'Zimbabwe': 'Zimbabwe', 'Wallis and Futuna': 'France', 'Italy': 'Italy', 'Timor-Leste': 'East Timor', 'Armenia': 'Armenia', 'French Southern Territories': 'France', 'Chad': 'Chad', 'El Salvador': 'El Salvador', 'Mozambique': 'Mozambique', 'Brunei Darussalam': 'Brunei', 'Morocco': 'Morocco', 'Saint Helena': 'Ascension and Tristan da Cunha', 'Portugal': 'Portugal', 'Central African Republic': 'Central African Republic', 'Seychelles': 'Seychelles', 'Dominica': 'Dominican Republic', 'Finland': 'Finland', 'Belgium': 'Belgium', 'Niue': 'Niue', 'Japan': 'Japan', 'Palestinian Territory': 'Palestinian Territories', 'Faroe Islands': 'Faroe Islands', 'Latvia': 'Latvia', 'Saudi Arabia': 'Saudi Arabia', 'Poland': 'Poland', 'Micronesia': 'Federated States of Micronesia', 'Macao': 'China', 'Bangladesh': 'Bangladesh', 'Taiwan': 'Taiwan', 'Slovakia (Slovak Republic)': 'Slovakia', 'Mexico': 'Mexico

In [40]:
df_Copy['Country'] = df_Copy['Country'].map(Normalized_Countries)

In [33]:
print(len(set(Normalized_Countries.values())))

216


In [41]:
df_Copy.head()

,Name,Last_Name,Email,Date,Country,YOE,Seniority,Technology,Code_Score,Interview_Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


In [51]:
grouped_df = df_Copy.groupby('Email').agg({
    'Technology': list,
    'Code_Score': list,
    'Interview_Score': list,
    'Email': 'size'
}).rename(columns={'Email': 'ApplicationsCount'}).reset_index()

filtered_df = grouped_df[grouped_df['ApplicationsCount'] > 1]

print(f"Number of reapplicants: {filtered_df.shape[0]}")
filtered_df.head(5)

Number of reapplicants: 165


,Email,Technology,Code_Score,Interview_Score,ApplicationsCount
93,abbigail94@yahoo.com,"[Client Success, DevOps]","[5, 6]","[8, 2]",2
420,addison_bode@hotmail.com,"[Development - Backend, Database Administration]","[7, 10]","[2, 3]",2
1137,alberta95@gmail.com,"[Salesforce, Adobe Experience Manager]","[9, 7]","[1, 2]",2
1163,alberto11@yahoo.com,"[Business Intelligence, Sales]","[2, 0]","[9, 4]",2
1286,alejandra17@hotmail.com,"[Development - Backend, Development - FullStack]","[6, 2]","[10, 9]",2


In [ ]:
# Drop duplicates
df_unique = df_Copy.drop_duplicates(subset='Email')

#transform the column 'Date' to datetime
df_unique['Date'] = pd.to_datetime(df_unique['Date'])

#extract the year from the date
df_unique['Date'] = df_unique['Date'].dt.year

#count the number of applications per year
applications_per_year = df_unique['Date'].value_counts().sort_index()

#show the plot
applications_per_year.plot(kind='bar')
plt.xlabel('Year')
plt.ylabel('Number of Applications')
plt.title('Number of Applications per Year')
plt.xticks(rotation=0)
plt.show()


In [ ]:
df_sorted = df_Copy.drop_duplicates(subset='Email')
#group by technology and count the number of applications
df_sorted = df_Copy.groupby('Technology').size().sort_values(ascending=False).reset_index(name='Count')

#create a color palette
green_yellow_palette = plt.cm.YlGn(np.linspace(0, 1, len(df_sorted['Technology'])))[::-1]

#create the bar plot
plt.figure(figsize=(15, 6))
bars = plt.barh(df_sorted['Technology'], df_sorted['Count'], color=green_yellow_palette)

#add the count values on top of the bars
for bar in bars:
    plt.annotate(f'{int(bar.get_width())}', 
                 xy=(bar.get_width(), bar.get_y() + bar.get_height() / 2),
                 ha='center', va='center', xytext=(20, 0), textcoords='offset points')

#set the labels and title
plt.xlabel('Count')
plt.ylabel('Technology')
plt.title('Count of Technologies')
plt.grid(False)
plt.show()

In [ ]:
fig, ax = plt.subplots()

#create the boxplot
ax.boxplot([df_Copy[df_Copy['Seniority'] == category]['YOE'] for category in df_Copy['Seniority'].unique()])

#customize the plot
ax.set_xticklabels(df_Copy['Seniority'].unique())
ax.set_xlabel('Seniority')
ax.set_ylabel('Years of Experience (YOE)')
ax.set_title('Distribution of Years of Experience by Seniority')

plt.show()